In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys


import time
import pandas as pd

### 설정
- **review_targets변수 name, url변경해서 실행!**

In [ ]:
# 크롬 드라이버 실행
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


# 수집할 리뷰 페이지 URL 목록
review_targets = [
    {'name':'방앗간_담음', 'url': "https://map.naver.com/p/search/%EB%8B%B4%EC%9D%8C%20%EB%B0%A9%EC%95%97%EA%B0%84/place/1159683139?c=15.00,0,0,0,dh&placePath=/review?entry=bmp&from=map&fromPanelNum=2&locale=ko&searchText=%EB%8B%B4%EC%9D%8C%20%EB%B0%A9%EC%95%97%EA%B0%84&svcName=map_pcv5&timestamp=202507222257&from=map&fromPanelNum=2&timestamp=202507222257&locale=ko&svcName=map_pcv5&searchText=%EB%8B%B4%EC%9D%8C%20%EB%B0%A9%EC%95%97%EA%B0%84"}, 
    {'name':'방앗간아이', 'url': 'https://map.naver.com/p/search/%EB%B0%A9%EC%95%97%EA%B0%84%EC%95%84%EC%9D%B4/place/36771708?c=14.78,0,0,0,dh&placePath=/review?entry=bmp&from=map&fromPanelNum=2&locale=ko&searchText=%EB%B0%A9%EC%95%97%EA%B0%84%EC%95%84%EC%9D%B4&svcName=map_pcv5&timestamp=202507222307&from=map&fromPanelNum=2&locale=ko&searchText=%EB%B0%A9%EC%95%97%EA%B0%84%EC%95%84%EC%9D%B4&svcName=map_pcv5&timestamp=202507222307'}, 
    {'name':'양촌방앗간', 'url': 'https://map.naver.com/p/search/%EB%8C%80%EC%A0%84%20%EB%B0%A9%EC%95%97%EA%B0%84/place/1882029547?c=10.00,0,0,0,dh&placePath=/review?entry=bmp&from=map&fromPanelNum=2&locale=ko&searchText=%EB%8C%80%EC%A0%84%20%EB%B0%A9%EC%95%97%EA%B0%84&svcName=map_pcv5&timestamp=202507221810&from=map&fromPanelNum=2&timestamp=202507221810&locale=ko&svcName=map_pcv5&searchText=%EB%8C%80%EC%A0%84%20%EB%B0%A9%EC%95%97%EA%B0%84'}, 
    {'name': '제일방앗간', 'url': 'https://map.naver.com/p/search/%EB%8C%80%EC%A0%84%20%EB%B0%A9%EC%95%97%EA%B0%84/place/19778591?c=10.00,0,0,0,dh&placePath=/review?entry=bmp&from=map&fromPanelNum=2&locale=ko&searchText=%EB%8C%80%EC%A0%84%20%EB%B0%A9%EC%95%97%EA%B0%84&svcName=map_pcv5&timestamp=202507221815&from=map&fromPanelNum=2&timestamp=202507221815&locale=ko&svcName=map_pcv5&searchText=%EB%8C%80%EC%A0%84%20%EB%B0%A9%EC%95%97%EA%B0%84'}
]
# 검색결과 iframe으로 전환
# driver.switch_to.frame("searchIframe")
# time.sleep(2)

### 실행

In [6]:
result_data = []
for target in review_targets:
    name = target["name"]
    url = target["url"]

    driver.get(url)
    time.sleep(3)

    try:
        # entryIframe 전환
        driver.switch_to.frame("entryIframe")
        time.sleep(2)

        # 리뷰탭 클릭
        try:
            a_tags = driver.find_elements(By.CSS_SELECTOR, "div.YYh8o.gHymq a")
            for a in a_tags:
                try:
                    span = a.find_element(By.CLASS_NAME, "veBoZ")
                    if span.text.strip() == "리뷰":
                        a.click()
                        time.sleep(3)
                        break
                except:
                    continue
        except:
            print(f"[{name}] 리뷰 탭 클릭 실패")
            continue

        # '리뷰 더보기' 반복 클릭
        while True:
            try:
                see_more_btn = driver.find_element(By.CLASS_NAME, 'fvwqf')
                see_more_btn.click()
                time.sleep(2)
            except:
                break

        # 펼치기 버튼 클릭 (있는 경우)
        try:
            driver.find_element(By.CLASS_NAME, 'YsfhA').click()
        except:
            pass

        # 리뷰 수집
        review_texts = []

        review_list = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "_review_list"))
        )
        review_items = review_list.find_elements(By.CLASS_NAME, "place_apply_pui")

        for li in review_items:
            try:
                text_div = li.find_element(By.CLASS_NAME, "pui__vn15t2")
                review_text = text_div.find_element(By.TAG_NAME, "a").text.strip()
                review_texts.append(review_text)
            except:
                continue

        print(f"[{name}] 리뷰 {len(review_texts)}개 수집 완료")
        result_data.append({
            "가게이름": name,
            "리뷰목록": review_texts
        })

    except Exception as e:
        print(f"[{name}] 에러 발생: {e}")
        continue

# 브라우저 종료
driver.quit()

# 저장
df = pd.DataFrame(result_data)
df.to_csv("가게리뷰_수집결과.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료!")


[시골기름집] 리뷰 358개 수집 완료
[대전기름집] 리뷰 246개 수집 완료
[양촌방앗간] 리뷰 312개 수집 완료
[제일방앗간] 리뷰 195개 수집 완료
CSV 저장 완료!


In [7]:
len(df)

4

In [8]:
df.head()

,가게이름,리뷰목록
0,시골기름집,"[굿, 친절해요, 감사합니다, 포장이 깔끔해 좋아요, 굿, 좋아요., 그래도 대기업..."
1,대전기름집,"[굿, 엄마단골 기름집입니다\n심부름으로 처음방문했는데 일하시는 젊은 분들이 친절하..."
2,양촌방앗간,[항상 가는곳이에요. 다른분들은 친절하고 좋았는데 오늘 판매하시는분은 일 하기 싫은...
3,제일방앗간,"[깔끔하고 건강한맛 넘~~좋아요, 사장님이 정말 친절하시고 물건에 품질이 좋아요 !..."
